# 02_data_transform.ipynb — Transformação de Dados

## 1. Sobre

### 1.1 Visão Geral

Este notebook consome o arquivo JSON gerado pelo notebook de extração (`01_data_extraction.ipynb`) e aplica inspeção, normalização de nomes de colunas, conversões de tipo e limpeza. O objetivo é produzir tabelas processadas e tipadas em `data/processed/` prontos para carga em banco de dados ou análises posteriores.

Organização do notebook:
- Imports e ambiente;
- Utilitários de normalização de nomes;
- Leitura segura do JSON bruto;
- Inspeção inicial e detecção de tipos;
- Conversões de tipos (datas, numéricos, booleanos, strings);
- Normalização e flatten de colunas aninhadas (listas de dicionários);
- Limpeza adicional (remoção de colunas esparsas, resumo de listas);
- Persistência em Parquet/JSON.

### 1.1 Visão Técnica

Este notebook carrega o JSON bruto produzido na etapa de extração e realiza transformações controladas e documentadas para produzir os artefatos processados. Abaixo estão as práticas aplicadas e o comportamento esperado em cada etapa.

Boas práticas aplicadas:
- Leitura tolerante (fallback quando `pd.read_json` falha);
- Normalização consistente de nomes (utilitário `normalize_columns`);
- Conversões de tipo explícitas com `errors='coerce'` para segurança;
- Redução de complexidade em colunas aninhadas (explode + `pd.json_normalize`);
- Persistência em `parquet` para eficiência e preservação de tipos.

In [1]:
import os
import sys
from pathlib import Path
import pandas as pd
import json
import re
from dotenv import load_dotenv

# Garante que a raiz do projeto esteja no sys.path (mesma abordagem do notebook 01)
project_root = Path(os.getcwd()).resolve().parents[0]
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

from src.normalizers.col_normalizer import normalize_cols_names, normalize_cols_multivalorada

load_dotenv()

True

## 2. Carrega Dados

Carregar os dados brutos extraidos no notebook `01_data_extraction.ipynb`. Fonte: `data/raw/dados_obras_gov.json`.

In [2]:
# Paths de entrada/saida
ROOT = project_root
RAW_PATH = ROOT / 'data' / 'raw' / 'dados_obras_gov.json'
PROCESSED_DIR = ROOT / 'data' / 'processed'
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

print('Raw path:', RAW_PATH)
print('Processed dir:', PROCESSED_DIR)

# Leitura segura do JSON: se o arquivo contem objetos vazios, json.load pode falhar em alguns casos, entao usamos pandas.read_json com orientacao por records
try:
    df = pd.read_json(RAW_PATH, orient='records')
except ValueError:
    # fallback: carrega como texto e ignora objetos invalidos
    with open(RAW_PATH, 'r', encoding='utf-8') as f:
        raw_text = f.read()
    data = json.loads(raw_text)
    df = pd.json_normalize(data)

print('Dados carregados: rows=', len(df))
df.head(3)


Raw path: E:\VS-Code\Lappis-PS\data\raw\dados_obras_gov.json
Processed dir: E:\VS-Code\Lappis-PS\data\processed
Dados carregados: rows= 834


,idUnico,nome,cep,endereco,descricao,funcaoSocial,metaGlobal,dataInicialPrevista,dataFinalPrevista,dataInicialEfetiva,...,observacoesPertinentes,isModeladaPorBim,dataSituacao,tomadores,executores,repassadores,eixos,tipos,subTipos,fontesDeRecurso
0,50379.53-54,DL - 304/2024 - Contratação de instituição par...,None,None,Contratação de instituição para execução de se...,Ampliação da capacidade de trafego visando a m...,Projetos Básicos e Executivos de Engenharia,2024-12-20,2027-12-05,None,...,None,0.0,2024-12-20,[],[{'nome': 'DEPARTAMENTO NACIONAL DE INFRAESTRU...,[],"[{'id': 3, 'descricao': 'Econômico'}]","[{'id': 25, 'descricao': 'Rodovia', 'idEixo': 3}]","[{'id': 4, 'descricao': 'Acessos Terrestres', ...","[{'origem': 'Federal', 'valorInvestimentoPrevi..."
1,42724.53-27,Escola Classe Crixá São Sebastião,None,None,"Construção de Escola em Tempo Integral, Escola...",A construção da nova escola beneficiará 977 es...,"Construção de Escola em Tempo Integral, Escola...",2024-09-02,2028-09-02,None,...,None,0.0,2025-09-05,[],[{'nome': 'SECRETARIA DE ESTADO DE EDUCACAO DO...,[{'nome': 'FUNDO NACIONAL DE DESENVOLVIMENTO D...,"[{'id': 4, 'descricao': 'Social'}]","[{'id': 46, 'descricao': 'Educação', 'idEixo':...","[{'id': 84, 'descricao': 'Educação', 'idTipo':...","[{'origem': 'Federal', 'valorInvestimentoPrevi..."
2,19970.53-78,Reajuste do Contrato 45/2021 - Contrução do Ce...,70.602-600,"SAIS Área Especial 3, Setor Policial Sul",Reajuste do Contrato 45/2021 - Construção do C...,Contribuir para a melhor formação dos bombeiro...,Construção de um novo centro de formação e de ...,2021-09-14,2024-08-28,None,...,None,0.0,2023-02-06,[],[{'nome': 'CORPO DE BOMBEIROS MILITAR DO DISTR...,[{'nome': 'CORPO DE BOMBEIROS MILITAR DO DISTR...,"[{'id': 1, 'descricao': 'Administrativo'}]","[{'id': 1, 'descricao': 'Segurança Pública', '...","[{'id': 59, 'descricao': 'Obras em Imóveis de ...","[{'origem': 'Federal', 'valorInvestimentoPrevi..."


## 3 Analise inicial
A seguir mostramos informações básicas e as colunas detectadas. Isso ajuda a decidir quais **transformações aplicar** **(datas, booleans, colunas aninhadas etc.)** e também a porcentagem de valores nulos em cada coluna.

In [3]:
# Informacoes basicas
print('shape:', df.shape)
print('Sample dtypes:')
print(df.dtypes.head(30))

# Percentual de valores nulos por coluna (ordenado)
na_frac = df.isna().mean().sort_values(ascending=False)
na_frac.head(30)



shape: (834, 31)
Sample dtypes:
idUnico                                object
nome                                   object
cep                                    object
endereco                               object
descricao                              object
funcaoSocial                           object
metaGlobal                             object
dataInicialPrevista                    object
dataFinalPrevista                      object
dataInicialEfetiva                     object
dataFinalEfetiva                       object
dataCadastro                           object
especie                                object
natureza                               object
naturezaOutras                         object
situacao                               object
descPlanoNacionalPoliticaVinculado     object
uf                                     object
qdtEmpregosGerados                     object
descPopulacaoBeneficiada               object
populacaoBeneficiada                   object
ob

dataFinalEfetiva                      0.994005
dataInicialEfetiva                    0.967626
observacoesPertinentes                0.857314
qdtEmpregosGerados                    0.820144
populacaoBeneficiada                  0.817746
descPopulacaoBeneficiada              0.814149
naturezaOutras                        0.754197
descPlanoNacionalPoliticaVinculado    0.676259
cep                                   0.526379
endereco                              0.483213
isModeladaPorBim                      0.290168
especie                               0.004796
dataInicialPrevista                   0.003597
dataFinalPrevista                     0.003597
metaGlobal                            0.000000
natureza                              0.000000
dataCadastro                          0.000000
funcaoSocial                          0.000000
idUnico                               0.000000
nome                                  0.000000
descricao                             0.000000
uf           

In [4]:
print(df["situacao"])

0      Cadastrada
1       Cancelada
2      Cadastrada
3      Cadastrada
4      Cadastrada
          ...    
829    Cadastrada
830    Cadastrada
831    Cadastrada
832    Cadastrada
833    Cadastrada
Name: situacao, Length: 834, dtype: object


## 4. Normalização

### 4.1 Conversão de Tipos

Identificamos as colunas que devem ser **convertidas para datas** `datetime64`, **inteiros** `float64`, **binárias** `boolean` e **texto** `string`. Foi utilizado um **mapeamento** baseado em nomes de coluna comuns detectados no dataset **para realizar as transformações** adequadas.

In [5]:
# Colunas de datas
date_columns = [
    'dataInicialPrevista', 'dataFinalPrevista', 'dataInicialEfetiva',
    'dataFinalEfetiva', 'dataCadastro', 'dataSituacao'
]
for col in date_columns:
    df[col] = pd.to_datetime(df[col], errors='coerce') # 'coerce' transforma erros em NaT (Not a Time)

# Colunas numéricas
numeric_columns = ['qdtEmpregosGerados', 'populacaoBeneficiada']
for col in numeric_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce') # 'coerce' transforma erros em NaN

# Coluna booleana
df['isModeladaPorBim'] = df['isModeladaPorBim'].astype('boolean')

text_columns = [
    'idUnico', 'nome', 'cep', 'endereco', 'descricao', 'funcaoSocial',
    'metaGlobal', 'especie', 'natureza', 'naturezaOutras', 'situacao',
    'descPlanoNacionalPoliticaVinculado', 'uf', 'descPopulacaoBeneficiada',
    'observacoesPertinentes'
]

for col in text_columns:
    # Verifique se a coluna existe antes de converter
    if col in df.columns:
        df[col] = df[col].astype('string') # A conversão acontece aqui!


print("\nTipos de Dados Corrigidos (DataFrame Principal):")
print(df.info())


Tipos de Dados Corrigidos (DataFrame Principal):
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 834 entries, 0 to 833
Data columns (total 31 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   idUnico                             834 non-null    string        
 1   nome                                834 non-null    string        
 2   cep                                 395 non-null    string        
 3   endereco                            431 non-null    string        
 4   descricao                           834 non-null    string        
 5   funcaoSocial                        834 non-null    string        
 6   metaGlobal                          834 non-null    string        
 7   dataInicialPrevista                 831 non-null    datetime64[ns]
 8   dataFinalPrevista                   831 non-null    datetime64[ns]
 9   dataInicialEfetiva                  27 non-null 

### 4.2 Colunas 

Separação das colunas aninhadas em seus próprios `DataFrame`. Além disso, está implementado a **transformação** da **descrição** ou **nome**, do tipo **Objeto** para `string`.

In [6]:
# Tabela principal (projetos) - vamos manter apenas as colunas "simples"
colunas_aninhadas = ['tomadores', 'executores', 'repassadores', 'eixos', 'tipos', 'subTipos', 'fontesDeRecurso']
df_projetos = df.drop(columns=colunas_aninhadas)

# Criando os DataFrames "filhos" (ignorei tomadores, muito semelhante ao executadores)
df_executores = normalize_cols_multivalorada(df, 'idUnico', 'executores')
df_eixos = normalize_cols_multivalorada(df, 'idUnico', 'eixos')
df_tipos = normalize_cols_multivalorada(df, 'idUnico', 'tipos')
df_subtipos = normalize_cols_multivalorada(df, 'idUnico', 'subTipos')
df_fontes_recurso =normalize_cols_multivalorada(df, 'idUnico', 'fontesDeRecurso')

# Transformando em string as colunas necessarias de cada DataFrame filho
df_executores["nome"] = df_executores["nome"].astype("string")
df_eixos["descricao"] = df_eixos["descricao"].astype("string")
df_tipos["descricao"] = df_tipos["descricao"].astype("string")
df_subtipos["descricao"] = df_subtipos["descricao"].astype("string")
df_fontes_recurso["origem"] = df_fontes_recurso["origem"].astype("string")

print("\n--- DataFrames Normalizados ---")
print("\nDataFrame de Projetos (Principal):")
print(df_projetos.head())

print("\nDataFrame de Executores:")
print(df_executores.head())

print("\nDataFrame de Fontes de Recurso:")
print(df_fontes_recurso.head())


--- DataFrames Normalizados ---

DataFrame de Projetos (Principal):
       idUnico                                               nome         cep  \
0  50379.53-54  DL - 304/2024 - Contratação de instituição par...        <NA>   
1  42724.53-27                  Escola Classe Crixá São Sebastião        <NA>   
2  19970.53-78  Reajuste do Contrato 45/2021 - Contrução do Ce...  70.602-600   
3  24797.53-15  Implantação de Passarelas nas Estradas Parque ...        <NA>   
4  24822.53-70  obra de construção da  Cabine de Medição, loca...        <NA>   

                                   endereco  \
0                                      <NA>   
1                                      <NA>   
2  SAIS Área Especial 3, Setor Policial Sul   
3                                      <NA>   
4                                      <NA>   

                                           descricao  \
0  Contratação de instituição para execução de se...   
1  Construção de Escola em Tempo Integral, Escola

### 4.3 Normalização dos Titulos das Colunas

Utilizando um módulo construido dentro de `src/normalizers/`, chamado `normalize_cols_names`, foi implementado a normalização dos tiutlos das colunas de cada DataFrame.

- Exemplo: idUnico -> id_unico

In [7]:
# Normalizando o titula das colunas
for df_i in [df_projetos, df_executores, df_eixos, df_tipos, df_subtipos, df_fontes_recurso]:
    df_i = normalize_cols_names(df_i, True)

### 4.4 Checagem 

Colunas agora com a **tipagem adequada** e **nomiação** correta, seguindo as boas práticas da ciência de dados.

In [9]:
df_projetos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 834 entries, 0 to 833
Data columns (total 24 columns):
 #   Column                                  Non-Null Count  Dtype         
---  ------                                  --------------  -----         
 0   id_unico                                834 non-null    string        
 1   nome                                    834 non-null    string        
 2   cep                                     395 non-null    string        
 3   endereco                                431 non-null    string        
 4   descricao                               834 non-null    string        
 5   funcao_social                           834 non-null    string        
 6   meta_global                             834 non-null    string        
 7   data_inicial_prevista                   831 non-null    datetime64[ns]
 8   data_final_prevista                     831 non-null    datetime64[ns]
 9   data_inicial_efetiva                    27 non-null   

In [10]:
df_executores.info()

<class 'pandas.core.frame.DataFrame'>
Index: 894 entries, 0 to 833
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id_unico  894 non-null    string 
 1   nome      890 non-null    string 
 2   codigo    890 non-null    float64
dtypes: float64(1), string(2)
memory usage: 27.9 KB


In [11]:
df_eixos.info() 

<class 'pandas.core.frame.DataFrame'>
Index: 989 entries, 0 to 833
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id_unico   989 non-null    string 
 1   id         985 non-null    float64
 2   descricao  985 non-null    string 
dtypes: float64(1), string(2)
memory usage: 30.9 KB


In [12]:
df_tipos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 989 entries, 0 to 833
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id_unico   989 non-null    string 
 1   id         985 non-null    float64
 2   descricao  985 non-null    string 
 3   id_eixo    985 non-null    float64
dtypes: float64(2), string(2)
memory usage: 38.6 KB


In [13]:
df_subtipos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 989 entries, 0 to 833
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id_unico   989 non-null    string 
 1   id         985 non-null    float64
 2   descricao  985 non-null    string 
 3   id_tipo    985 non-null    float64
dtypes: float64(2), string(2)
memory usage: 38.6 KB


In [14]:
df_fontes_recurso.info()

<class 'pandas.core.frame.DataFrame'>
Index: 837 entries, 0 to 833
Data columns (total 3 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   id_unico                     837 non-null    string 
 1   origem                       837 non-null    string 
 2   valor_investimento_previsto  837 non-null    float64
dtypes: float64(1), string(2)
memory usage: 26.2 KB


## 5. Limpeza adicional e remoção de colunas com muitos valores nulos
Removeremos colunas com **mais de 95% de valores nulos** — são geralmente não informativas para análise inicial. Também vamos achatar colunas de listas quando possível (ex.: tomadores, executores) mantendo o comprimento máximo ou a primeira entrada.

In [ ]:
# Remover colunas com frac de NA acima de um limiar
SPARSE_THRESHOLD = 0.95
na_frac = df.isna().mean()
sparse_cols = na_frac[na_frac > SPARSE_THRESHOLD].index.tolist()
print('Colunas muito esparsas (serao removidas):', len(sparse_cols))

df = df.drop(columns=sparse_cols)

list_cols = [c for c in df.columns if df[c].apply(lambda x: isinstance(x, list)).any()]
print('Colunas que contêm listas (serao resumidas):', list_cols)
for c in list_cols:
    # Converte lista vazia ou nao-lista para None, senao pega o primeiro elemento ou comprimento
    df[c + '_len'] = df[c].apply(lambda x: len(x) if isinstance(x, list) else (0 if pd.isna(x) else 1))
    # Também extrai o primeiro item se existir e for dict com 'nome' ou 'id'
    def first_summary(v):
        if isinstance(v, list) and len(v) > 0:
            first = v[0]
            if isinstance(first, dict):
                return first.get('nome') or first.get('id') or str(first)
            return first
        return pd.NA
    df[c + '_first'] = df[c].apply(first_summary)
    # drop original list column para evitar campos complexos
    df = df.drop(columns=[c])

print('Shape apos limpeza:', df.shape)
df.head(2)


Colunas muito esparsas (serao removidas): 4


Colunas que contêm listas (serao resumidas): ['tomadores', 'executores', 'repassadores', 'eixos', 'tipos', 'subTipos', 'fontesDeRecurso']
Shape apos limpeza: (834, 34)


,idUnico,nome,cep,endereco,descricao,funcaoSocial,metaGlobal,dataInicialPrevista,dataFinalPrevista,dataCadastro,...,repassadores_len,repassadores_first,eixos_len,eixos_first,tipos_len,tipos_first,subTipos_len,subTipos_first,fontesDeRecurso_len,fontesDeRecurso_first
0,50379.53-54,DL - 304/2024 - Contratação de instituição par...,<NA>,<NA>,Contratação de instituição para execução de se...,Ampliação da capacidade de trafego visando a m...,Projetos Básicos e Executivos de Engenharia,2024-12-20,2027-12-05,2024-12-20,...,0,<NA>,1,3,1,25,1,4,1,"{'origem': 'Federal', 'valorInvestimentoPrevis..."
1,42724.53-27,Escola Classe Crixá São Sebastião,<NA>,<NA>,"Construção de Escola em Tempo Integral, Escola...",A construção da nova escola beneficiará 977 es...,"Construção de Escola em Tempo Integral, Escola...",2024-09-02,2028-09-02,2024-08-30,...,1,FUNDO NACIONAL DE DESENVOLVIMENTO DA EDUCAÇÃO,1,4,1,46,1,84,1,"{'origem': 'Federal', 'valorInvestimentoPrevis..."


## 6. Salvando saída processada
Salvamos o resultado em `data/processed/` em Parquet por ser **compacto** e, especialmente, **preserva tipos**.

In [16]:
df_projetos.to_parquet(
    str(PROCESSED_DIR) + '/dados_projetos_limpos.parquet', 
    engine='pyarrow',
    index=False
)
df_executores.to_parquet(
    str(PROCESSED_DIR) + '/dados_executores_limpos.parquet', 
    engine='pyarrow',
    index=False
)
df_eixos.to_parquet(
    str(PROCESSED_DIR) + '/dados_eixos_limpos.parquet', 
    engine='pyarrow',
    index=False
)
df_subtipos.to_parquet(
    str(PROCESSED_DIR) + '/dados_subtipos_limpos.parquet', 
    engine='pyarrow',
    index=False
)
df_fontes_recurso.to_parquet(
    str(PROCESSED_DIR) + '/dados_fontes_recurso_limpos.parquet', 
    engine='pyarrow',
    index=False
)


### Observações finais
- Ajustes adicionais (imputação, encoding categórico, feature engineering) devem ser feitos em notebooks posteriores com base nas necessidades da análise.
- Mantido um arquivo Parquet que preserva tipos e é eficiente para cargas futuras.